In [33]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [34]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [38]:
import folium
import pandas as pd
from folium.plugins import HeatMap
from geopy.geocoders import Nominatim

In [39]:
def latitude(n):
    geolocator = Nominatim(user_agent="Planet Health")
    location = geolocator.geocode(n)
    
    if location is None:
        return None

    return float(location.latitude) if location.latitude is not None else None if location is not None else None

def longitude(n):
    geolocator = Nominatim(user_agent="Planet Health")
    location = geolocator.geocode(n)
    
    if location is None:
        return ""

    return float(location.longitude) if location.longitude is not None else None if location is not None else None

In [41]:
df_voos = pd.read_csv("files/VRA_102019.csv", delimiter=";", usecols=['sg_icao_origem', 'sg_icao_destino', 'dt_partida_real'])
df_aeroportos = pd.read_csv("files/aeroportos.csv", delimiter=";")
df_voos = df_voos.rename(columns={"sg_icao_origem":"sg_origem", "sg_icao_destino":"sg_destino", "dt_partida_real":"data"})

In [42]:
df_origen = df_aeroportos.rename(columns={"Sigla OACI":"sg_origem", "Descrição":"desc", "Cidade":"cidade_origem", "UF":"uf", "País":"pais_origem", "Continente":"continente"})
result = df_voos.merge(df_origen, on='sg_origem')
result = result[['sg_origem', 'cidade_origem', 'pais_origem', 'sg_destino', 'data']]

In [43]:
df_destino = df_aeroportos.rename(columns={"Sigla OACI":"sg_destino", "Descrição":"desc", "Cidade":"cidade_destino", "UF":"uf", "País":"pais_destino", "Continente":"continente"})
df_destino.head()
result = result.merge(df_destino, on='sg_destino')

In [44]:
result = result[['sg_origem', 'cidade_origem', 'pais_origem', 'sg_destino', 'cidade_destino', 'pais_destino', 'data']]
result['data'] = result['data'].str[:10]

In [45]:
voos_dia = pd.DataFrame({'voos' : result.groupby(["cidade_origem", "cidade_destino", "data"]).size()}).reset_index()
voos_dia = voos_dia.sort_values(by=['data']).reset_index()


In [46]:
geolocator = Nominatim(user_agent="Planet Health")
vo = pd.DataFrame(data=voos_dia['cidade_destino'].unique())

In [31]:
la = pd.DataFrame({"cidade_destino": vo[0], 'latitude' : vo[0].map(latitude), 'longitude' : vo[0].map(longitude)})

KeyboardInterrupt: 

In [ ]:
a = voos_dia.merge(la, on='cidade_destino')

In [ ]:
dados_map = a[['latitude','longitude', 'voos']].dropna()
dados_map = dados_map.rename(columns={'latitude':'lat','longitude':'lng', 'voos':'weight'})
dados_map.sort_values(by=['weight']).reset_index()

In [ ]:
geolocator = Nominatim(user_agent="Planet Health")
location = geolocator.geocode("curitiba")

mapa = folium.Map(location=[location.latitude, location.longitude])
HeatMap(data=dados_map.values.tolist()).add_to(mapa)
mapa